In [1]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers

In [2]:
start_time = time.time()

In [3]:
preproc.do_preprocessing('../twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../twitter-datasets/train_neg.txt')

100%|██████████| 100000/100000 [04:16<00:00, 390.06it/s]


In [82]:
with open('../twitter-datasets/train_pos_full_processed.txt') as pos_in, open(
                            '../twitter-datasets/train_neg_full_processed.txt') as neg_in:
    pos_lines = pos_in.readlines()
    neg_lines = neg_in.readlines()
    pos_in.close()
    neg_in.close()
lines = pos_lines + neg_lines
lines = [line.replace('\n', '') for line in lines]

In [83]:
print(len(lines))
lines[:5]

2500000


['duno justin read mention . onli justin god know , hope follow #believ',
 'becaus logic dumb , even crop name photo . tsk .',
 '" put casper box ! " love battl ! #crakbitch',
 'thank sir > > trip lil mama . keep doin ya thang !',
 'visit brother tmr bestest birthday gift ever ! ! !']

In [84]:
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer(lowercase=True , norm='l2')
X = vectorizer.fit_transform(lines)

y = np.zeros(shape=(len(lines)))
y[:len(pos_lines)] = 1
y[len(pos_lines):] = -1

In [85]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# clf = LogisticRegression().fit(X, y)
forest = RandomForestClassifier(n_estimators=50, max_depth=50, random_state=4, n_jobs=-1)

kf = KFold(n_splits=4, shuffle=True, random_state=0)
scores = cross_val_score(forest, X, y, cv=kf)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
preproc.do_preprocessing('../twitter-datasets/test_data.txt')

In [ ]:
with open('../twitter-datasets/test_data_processed.txt') as test_in:
    test_lines = test_in.readlines()
test_lines = [line.replace('\n', '') for line in test_lines]

In [ ]:
print(len(test_lines))
print(test_lines[:5])

In [ ]:
X_test = vectorizer.transform(test_lines)

y_pred = clf.predict(X_test)

helpers.create_submission_csv(y_pred)

In [ ]:
print(np.unique(y_pred))

In [ ]:
elapsed_time = divmod(round((time.time() - start_time)), 60)
print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))

In [ ]:
pos_lines[1]

In [ ]:
preproc.remove_stopwords(pos_lines[0])